# Notebook for simulating, saving and analysing MOMA-like data with "fuzzy" initital parameters


In [4]:
from cells_simulation import *

In [5]:
# ========== Simulation parameters ========== #
save_dataset = True    # saves data set as csv
run_ggp = True         # runs the ggp code (located in "../bin/" relative to this nb)

dt = 1e-3
dt_measument = 3 # in minutes
n_cells = 500 # number of cells that will be simulated

div_mode = "sizer"
division_log_length = 1+np.log(2)   # for sizer: division, when log_length hits division_log_length
division_time = 60 - 1e-10          # for timer: division, when cell cycle time hits division_time
division_addition = np.log(2)       # for adder: divsion, when division_addition in log_length was added in cell cycle

# ========== Model parameters ========== #
parameter_set = {   "mean_lambda": 0.01, 
                    "gamma_lambda": 0.01,
                    "var_lambda": 1e-7,
                    "mean_q": 10,
                    "gamma_q": 1e-2,
                    "var_q": 0.1,
                    "beta": 0.001, #1e-3,
                    "var_x": 1e-3,
                    "var_g": 5000,
                    "var_dx": 0.001,
                    "var_dg": 500}

In [6]:


# # --------- SIMULATE --------- #
# cells_simulated = simulate_cells(dt, n_cells, parameter_set, div_mode,
#                          division_log_length, 
#                          division_time, 
#                          division_addition)

# dataset = build_data_set(cells_simulated, parameter_set['var_x'], parameter_set['var_g'], int(dt_measument/dt))


In [21]:
def write_param_file(filename, parameters, non_default={}):
    with open(filename, "w") as fin:
        fin.write("# Generated config file for simulated data\n")
        for k, v in parameters.items():
            fin.write("{:s} = {:.2E}, {:.2E}\n".format(k, v, v/.))

In [22]:
def add_gaussian_noise2params(parameters, rel_noise=0.1, rel_min=0.01):
    new = copy.deepcopy(parameters)
    for k, v in new.items():
        temp = new[k] + np.random.normal(loc=0, scale=new[k]*rel_noise)
        new[k] = np.max([temp, v*rel_min])
    return new

non_default = dict()

# col = []
# for i in range(5000):
#     col.append(add_uni_noise2params(parameter_set)["var_g"])

# plt.hist(col, 100)

In [53]:
def suggest_run_command(directory, filename, modes ="-s -m -p", out_dir=None):
    cmd = "../bin/gfp_gaussian -c " + os.path.join(directory, "csv_config.txt") + \
        " -b " + os.path.join(directory, "parameters.txt") + \
        " -t 1e0 -i " + filename + " -l 0 "
    if out_dir==None:
        return cmd + modes
    else:
        return cmd + modes + " -o " + out_dir


In [52]:
out_dir = mk_mising_dir('../data_2.0/fuzzy_param_guess_sizer')

noise_levels = [0.5 for i in range(10)]
print(*noise_levels, "\n")

for i in range(10):
    # ----------- SAVE ----------- #
    if save_dataset:
        print(i)
        directory, filename = get_next_file_name(out_dir, force_index=i)

        # fuzzy_params = add_gaussian_noise2params(parameter_set, param_noise)
        # write_param_file(os.path.join(directory, "parameters.txt"), fuzzy_params, non_default) 

        # write_csv_config(os.path.join(directory, "csv_config.txt"))
        # dataset.to_csv(filename)
        print("New simulation saved in", filename)

        # ----------- RUN ----------- #
        if run_ggp:
            run_cmd = suggest_run_command(directory, filename, modes="-m -p", out_dir=os.path.join(directory, "LN_COBYLA"))
            print("run: -> \n", run_cmd)
            s = subprocess.run(run_cmd, capture_output=True, shell=True)
            print(s)
        

0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 

0
0
force i= 0
New simulation saved in ../data_2.0/fuzzy_param_guess_sizer/fuzzy_param_guess_sizer_0/fuzzy_param_guess_sizer_0.csv
run: -> 
 ../bin/gfp_gaussian -c ../data_2.0/fuzzy_param_guess_sizer/fuzzy_param_guess_sizer_0/csv_config.txt -b ../data_2.0/fuzzy_param_guess_sizer/fuzzy_param_guess_sizer_0/parameters.txt -t 1e-1 -i ../data_2.0/fuzzy_param_guess_sizer/fuzzy_param_guess_sizer_0/fuzzy_param_guess_sizer_0.csv -l 0 -m -p -o ../data_2.0/fuzzy_param_guess_sizer/fuzzy_param_guess_sizer_0/LN_COBYLA


KeyboardInterrupt: 